In [1]:
import pandas as pd
import re
import numpy as np
import sqlalchemy
import sqlite3

In [2]:
nodes = pd.read_csv('network/twitternetworknodes.csv')

In [3]:
verified_original = pd.read_json('/Users/briansrebrenik/DSCourseMaterials/projects/finalProject/twitterData/TU_verified.ndjson', lines=True)

In [5]:
verified_original.columns

Index(['contributors_enabled', 'created_at', 'default_profile',
       'default_profile_image', 'description', 'entities', 'favourites_count',
       'follow_request_sent', 'followers_count', 'following', 'friends_count',
       'geo_enabled', 'has_extended_profile', 'id', 'id_str',
       'is_translation_enabled', 'is_translator', 'lang', 'listed_count',
       'location', 'name', 'notifications', 'profile_background_color',
       'profile_background_image_url', 'profile_background_image_url_https',
       'profile_background_tile', 'profile_banner_url', 'profile_image_url',
       'profile_image_url_https', 'profile_link_color',
       'profile_sidebar_border_color', 'profile_sidebar_fill_color',
       'profile_text_color', 'profile_use_background_image', 'protected',
       'retrieved_on', 'screen_name', 'statuses_count', 'time_zone',
       'translator_type', 'url', 'utc_offset', 'verified',
       'withheld_in_countries'],
      dtype='object')

In [4]:
verified_original.set_index('id', inplace=True)

In [167]:
nodes.columns

Index(['Id', 'Label', 'timeset', 'listed', 'friends', 'followers', 'user_id',
       'language', 'modularity_class'],
      dtype='object')

In [5]:
combined = nodes.set_index('user_id').join(verified_original[['url', 'location', 'description', 'created_at', 'name']])

In [169]:
len(combined)

262803

In [170]:
combined.columns

Index(['Id', 'Label', 'timeset', 'listed', 'friends', 'followers', 'language',
       'modularity_class', 'url', 'location', 'description'],
      dtype='object')

In [6]:
combined.rename(columns={'Id': 'node_id'}, inplace=True)
combined.drop(columns=['timeset', 'listed', 'modularity_class'], inplace=True)
combined.head()

,node_id,Label,friends,followers,language,url,location,description,created_at,name
user_id,,,,,,,,,,
15736267,28835,LilianPacce,658,639276,en,http://t.co/v44qPcZZnz,São Paulo,"Editora de moda, apresentadora, jornalista e e...",2008-08-05 14:06:35,LilianPacce
313525999,204136,FenitN,1198,3160,en,https://t.co/c8LNamHxLp,"Washington, DC",I cover D.C. government for @WashingtonPost. S...,2011-06-08 19:57:08,Fenit Nirappil
34177979,249077,eringilfoy,362,78126,en,https://t.co/AvxYBz13qS,"Los Angeles, CA",instagram: @eringilfoy snapchat: @erin_gilfoy ...,2009-04-22 04:09:53,erin gilfoy
1774511,111815,stuartdredge,1295,22945,en,https://t.co/SrR9EhvTO4,Essex,Scribbler: Music Ally / Observer / The Week Ju...,2007-03-21 17:42:25,Stuart Dredge
17284200,36410,hmvtweets,2115,124810,en,https://t.co/yEAVnZrvqQ,UK,home of entertainment since 1921. Rewards: @pu...,2008-11-10 09:41:22,hmv


In [18]:
df = pd.read_csv('/Users/briansrebrenik/DSCourseMaterials/projects/finalProject/twitterData/tweet_df_final.csv')

In [32]:
len(df)

61836077

In [23]:
verified_user_names = list(combined.Label)

In [25]:
verified_user_names = set(verified_user_names)

In [26]:
authors = df.author

In [27]:
from tqdm import tqdm
verified_tweets = []
for i in tqdm(range(len(authors))):
    if authors[i] in verified_user_names:
        verified_tweets.append(i)
    else:
        pass

100%|██████████| 61836077/61836077 [34:29<00:00, 29876.75it/s]  


In [28]:
verified_tweet_set = set(verified_tweets)

In [29]:
len(verified_tweet_set)

1172338

In [31]:
df.drop(columns=['Unnamed: 0', 'Unnamed: 0.1'], inplace=True)

In [33]:
new_rows= []
for i in tqdm(range(len(df))):
    if i in verified_tweet_set:
        new_rows.append(df.iloc[i])

100%|██████████| 61836077/61836077 [06:15<00:00, 164578.05it/s] 


In [34]:
verified_tweet_df = pd.DataFrame(new_rows)

In [36]:
len(verified_tweet_df)

1172338

In [37]:
#verified_tweet_df.to_csv('/Users/briansrebrenik/DSCourseMaterials/projects/finalProject/verified_tweet_df')

In [38]:
verified_tweet_df.head()

,author,date,topic_id,tweet_id
10,SilverbirdTV,2014-10-16 14:39:56,0,522758848529321984
25,BT_Vancouver,2014-10-16 14:37:38,0,522758268205432832
31,HelloCanada,2014-10-16 14:37:13,0,522758164400599040
37,Brimastroianni,2014-10-16 14:36:16,0,522757924234338306
46,NiraArora,2014-10-16 14:34:33,0,522757491743285248


In [2]:
df = pd.read_csv('verified_tweet_df')

In [24]:
df.drop(columns='Unnamed: 0', inplace=True)
df.set_index('author', inplace=True)
df.head()

,date,topic_id,tweet_id
author,,,
SilverbirdTV,2014-10-16 14:39:56,0,522758848529321984
BT_Vancouver,2014-10-16 14:37:38,0,522758268205432832
HelloCanada,2014-10-16 14:37:13,0,522758164400599040
Brimastroianni,2014-10-16 14:36:16,0,522757924234338306
NiraArora,2014-10-16 14:34:33,0,522757491743285248


In [45]:
engine = sqlalchemy.create_engine('sqlite:///tweetdatabase.db')
conn = engine.connect()

In [46]:
score_table = pd.read_sql_table('events', conn)

In [48]:
score_table.head()

,id,topic_terms,tweet_count,avg_score
0,0,"host,patrick,neil",34694,2.000000
1,1,"royals,game,series",22111,1.833333
2,2,"giants,game,win",9990,1.900000
3,3,"october,ebola,house",147,1.066667
4,4,"ebola,white,health",2956,1.633333


In [69]:
df_with_scores = df.reset_index().set_index('topic_id').join(score_table[['avg_score']])

In [70]:
df_with_scores.reset_index(inplace=True)


In [184]:
df_with_scores.rename(columns={'author': "user"}, inplace=True)

In [191]:
df_with_scores.set_index('tweet_id', inplace=True)
df_with_scores.head()

,topic_id,user,date,avg_score
tweet_id,,,,
522758848529321984,0,SilverbirdTV,2014-10-16 14:39:56,2.0
522758268205432832,0,BT_Vancouver,2014-10-16 14:37:38,2.0
522758164400599040,0,HelloCanada,2014-10-16 14:37:13,2.0
522757924234338306,0,Brimastroianni,2014-10-16 14:36:16,2.0
522757491743285248,0,NiraArora,2014-10-16 14:34:33,2.0


In [7]:
combined.rename(columns={'Label' : 'user'}, inplace=True)

In [8]:
combined.reset_index(inplace=True)
combined.set_index('user', inplace=True)
combined.head()

,user_id,node_id,friends,followers,language,url,location,description,created_at,name
user,,,,,,,,,,
LilianPacce,15736267,28835,658,639276,en,http://t.co/v44qPcZZnz,São Paulo,"Editora de moda, apresentadora, jornalista e e...",2008-08-05 14:06:35,LilianPacce
FenitN,313525999,204136,1198,3160,en,https://t.co/c8LNamHxLp,"Washington, DC",I cover D.C. government for @WashingtonPost. S...,2011-06-08 19:57:08,Fenit Nirappil
eringilfoy,34177979,249077,362,78126,en,https://t.co/AvxYBz13qS,"Los Angeles, CA",instagram: @eringilfoy snapchat: @erin_gilfoy ...,2009-04-22 04:09:53,erin gilfoy
stuartdredge,1774511,111815,1295,22945,en,https://t.co/SrR9EhvTO4,Essex,Scribbler: Music Ally / Observer / The Week Ju...,2007-03-21 17:42:25,Stuart Dredge
hmvtweets,17284200,36410,2115,124810,en,https://t.co/yEAVnZrvqQ,UK,home of entertainment since 1921. Rewards: @pu...,2008-11-10 09:41:22,hmv


In [9]:
conn = sqlite3.connect('tweetdatabase.db')
c = conn.cursor()

In [195]:
df_with_scores.to_sql('verified_tweets', conn)

In [10]:
combined.to_sql('verified_users', conn, if_exists='replace')